# भाग 8 - योजनाओं का परिचय


### प्रसंग

अब जब हम योजनाओं के माध्यम से आ गए हैं, तो हम प्रोटोकॉल नामक एक नई वस्तु पेश करेंगे। एक प्रोटोकॉल योजनाओं के अनुक्रम का समन्वय करता है, उन्हें दूर के श्रमिकों पर तैनात करता है और उन्हें एक ही पास में चलाता है।

यह एक उच्च स्तरीय वस्तु है जिसमें कई श्रमिकों में वितरित जटिल संगणना का तर्क शामिल है। प्रोटोकॉल की मुख्य विशेषता श्रमिकों के बीच वापस भेजे जाने / खोजे जाने और प्राप्त करने की क्षमता है, और अंत में पहचाने गए श्रमिकों के लिए तैनात की जाती है। इसलिए एक उपयोगकर्ता एक प्रोटोकॉल डिज़ाइन कर सकता है, इसे क्लाउड वर्कर पर अपलोड कर सकता है, और कोई भी अन्य श्रमिक इसके लिए खोज कर सकता है, इसे डाउनलोड कर सकता है, और गणना कार्यक्रम को उन श्रमिकों पर लागू कर सकता है जिनसे यह जुड़ा हुआ है।

आइए देखें कि इसका उपयोग कैसे करें!

लेखक:

    थThéo Ryffel - Twitter @theoryffel - GitHub: @LaRiffle

### आयात और मॉडल विनिर्देशों

पहले आधिकारिक आयात करते हैं।

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

And than those specific to PySyft, with one important note: **the local worker should not be a client worker.** *Non client workers can store objects and we need this ability to run a plan.*

और PySyft के लिए उन लोगों की तुलना में, एक महत्वपूर्ण नोट के साथ: **स्थानीय कार्यकर्ता ग्राहक नहीं होना चाहिए।** *गैर ग्राहक श्रमिक वस्तुओं को स्टोर कर सकते हैं और हमें योजना चलाने के लिए इस क्षमता की आवश्यकता है।*

In [ ]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

हम संदर्भ लेख में प्रदान की गई धारणाओं के अनुरूप होने के लिए, दूरस्थ श्रमिकों या _उपकरणों_ को परिभाषित करते हैं।
हम उन्हें कुछ डेटा प्रदान करते हैं।

In [ ]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

### मूल उदाहरण

आइए एक फ़ंक्शन को परिभाषित करें जिसे हम एक योजना में बदलना चाहते हैं। ऐसा करने के लिए, यह फ़ंक्शन परिभाषा के ऊपर एक डेकोरेटर जोड़ने जैसा सरल है!

In [ ]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

चलिए चेक करते हैं, हाँ अब हमारे पास एक योजना है!

In [ ]:
plan_double_abs

किसी योजना का उपयोग करने के लिए, आपको दो चीजों की आवश्यकता होती है: योजना बनाने के लिए (_अर्थात फ़ंक्शन में मौजूद संचालन के अनुक्रम को पंजीकृत करें_) और इसे एक कार्यकर्ता / उपकरण को भेजें। सौभाग्य से आप यह बहुत आसानी से कर सकते हैं!

#### एक योजना का निर्माण

एक प्लान बनाने के लिए आपको बस कुछ डेटा पर कॉल करना होगा।

आइए पहले कुछ दूरस्थ डेटा का संदर्भ प्राप्त करें: एक अनुरोध नेटवर्क पर भेजा जाता है और एक संदर्भ पॉइंटर लौटाया जाता है।

In [ ]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

अगर हम योजना को बताते हैं तो इसे डिवाइस पर दूरस्थ रूप से निष्पादित किया जाना चाहिए `location:device_1` ... हम एक त्रुटि प्राप्त करेंगे क्योंकि योजना अभी तक नहीं बनाई गई थी।

In [ ]:
plan_double_abs.is_built

In [ ]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

एक योजना बनाने के लिए आपको योजना पर `build` कॉल करने की आवश्यकता है और योजना को निष्पादित करने के लिए आवश्यक तर्क पास करें (कुछ डेटा a.k.a)। जब एक योजना का निर्माण किया जाता है तो सभी कमांड स्थानीय कार्यकर्ता द्वारा क्रमिक रूप से निष्पादित किए जाते हैं, और योजना द्वारा पकड़े जाते हैं और अपने `readable_plan` विशेषता में संग्रहीत होते हैं!

In [ ]:
plan_double_abs.build(torch.tensor([1., -2.]))

In [ ]:
plan_double_abs.is_built

अगर हम योजना को भेजने की कोशिश करते हैं तो यह काम करता है!

In [ ]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

टेंसर्स जैसे, हमें भेजे गए ऑब्जेक्ट को एक पॉइंटर मिलता है। यहाँ इसे बस एक `PointerPlan` कहा जाता है।

याद रखने के लिए एक महत्वपूर्ण बात यह है कि जब एक योजना बनाई जाती है तो हम आईडी (एस) के आगे पूर्व निर्धारित करते हैं जहां परिणाम (ओं) को संग्रहीत किया जाना चाहिए। यह आदेशों को अतुल्यकालिक रूप से भेजने की अनुमति देगा, पहले से ही एक आभासी परिणाम के संदर्भ में और दूरस्थ परिणाम की प्रतीक्षा किए बिना स्थानीय गणना जारी रखने के लिए। एक प्रमुख अनुप्रयोग है जब आपको device_1 पर एक बैच की गणना की आवश्यकता होती है और device_2 पर दूसरे बैच की गणना शुरू करने के लिए इस गणना के समाप्त होने का इंतजार नहीं करना चाहता।

#### दूर से एक योजना चला रहा है

अब हम प्‍वाइंटर प्‍लान पर कॉल करके को कुछ डेटा के साथ प्‍लान को रिमोट रन कर सकते हैं । यह इस योजना को दूरस्थ रूप से चलाने के लिए एक आदेश जारी करता है, ताकि योजना के आउटपुट का पूर्वनिर्धारित स्थान अब परिणाम को शामिल करें (याद रखें कि हम गणना से पहले परिणाम के पूर्व-निर्धारित स्थान को याद करते हैं)। इसके लिए एकल संचार दौर की भी आवश्यकता होती है।

परिणाम बस एक सूचक है, जैसे जब आप एक सामान्य हुक torch function चलाते हैं!

In [ ]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

और आप बस मान वापस पूछ सकते हैं।

In [ ]:
pointer_to_result.get()

### एक ठोस उदाहरण की ओर

लेकिन हम जो करना चाहते हैं, वह योजना को गहरी और संघबद्ध शिक्षा पर लागू करना है, है ना? तो चलो थोड़ा और अधिक जटिल उदाहरण देखें, neural नेटवर्क का उपयोग करते हुए जैसा कि आप उन्हें उपयोग करने के लिए तैयार हो सकते हैं।
ध्यान दें कि अब हम एक वर्ग को एक योजना में बदल रहे हैं। ऐसा करने के लिए, हम अपनी कक्षा को sy.Plan (nn.Module से विरासत में लेने के बजाय) से वारिस करते हैं।

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)


In [ ]:
net = Net()

In [ ]:
net

आइए कुछ मॉक डेटा का उपयोग करके प्‍लान का निर्माण करें।

In [ ]:
net.build(torch.tensor([1., 2.]))

अब हम एक दूरस्थ कार्यकर्ता को प्‍लान भेजते हैं

In [ ]:
pointer_to_net = net.send(device_1)
pointer_to_net

चलो कुछ दूरस्थ डेटा पुनर्प्राप्त करते हैं

In [ ]:
pointer_to_data = device_1.search('input_data')[0]

फिर, वाक्यविन्यास सामान्य दूरस्थ अनुक्रमिक निष्पादन की तरह है, अर्थात, स्थानीय निष्पादन की तरह। लेकिन क्लासिक रिमोट निष्पादन की तुलना में, प्रत्येक निष्पादन के लिए एक एकल संचार दौर है।

In [ ]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

और हम हमेशा की तरह परिणाम प्राप्त करते हैं!

In [ ]:
pointer_to_result.get()

Et voilà! हमने देखा है कि स्थानीय कार्यकर्ता (या सर्वर) और दूरस्थ उपकरणों के बीच संचार को नाटकीय रूप से कैसे कम करें!

### कार्यकर्ताओं के बीच स्विच करें

एक प्रमुख विशेषता जो हम चाहते हैं, वह यह है कि कई श्रमिकों के लिए एक ही योजना का उपयोग किया जाए, जिस डेटा पर हम विचार कर रहे हैं, उसके आधार पर हम बदलेंगे।
विशेष रूप से, हम हर बार जब हम कार्यकर्ता बदलते हैं तो योजना का पुनर्निर्माण नहीं करना चाहते हैं। आइए देखें कि हम अपने छोटे नेटवर्क के साथ पिछले उदाहरण का उपयोग करके यह कैसे करते हैं।

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [ ]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

यहां मुख्य कदम हैं जिन्हें हमने निष्पादित किया है

In [ ]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

और वास्तव में आप एक ही योजना से अन्य PointerPlans का निर्माण कर सकते हैं, इसलिए सिंटैक्स किसी अन्य डिवाइस की योजना को दूरस्थ रूप से चलाने के लिए समान है

In [ ]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

> नोट: वर्तमान में, योजना वर्गों के साथ, आप केवल एक विधि का उपयोग कर सकते हैं और आपको इसे "forward" नाम देना होगा।

### फ़ंक्शंस हैं स्वचालित रूप से योजनाओं का निर्माण

फ़ंक्शंस के लिए (`@` `sy.func2plan`) हम प्‍लान को स्पष्ट रूप से `build` कहने की आवश्यकता नहीं के साथ स्वचालित रूप से बना सकते हैं, वास्तव में योजना के निर्माण के क्षण में पहले से ही बनाया गया है।

इस कार्यक्षमता को प्राप्त करने के लिए केवल एक योजना बनाते समय आपको बदलने की आवश्यकता होती है, डेकोरेटर को `args_shape` नामक एक तर्क सेट किया जाता है, जिसमें प्रत्येक तर्क के आकार वाली एक सूची होनी चाहिए।

In [ ]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

`Args_shape` पैरामीटर का उपयोग आंतरिक रूप से दिए गए आकार के साथ नकली टेंसर बनाने के लिए किया जाता है जो प्‍लान बनाने के लिए उपयोग किए जाते हैं।

In [ ]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

आप कार्यों को state elements भी प्रदान कर सकते हैं!

In [ ]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [ ]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

इस बारे में अधिक जानने के लिए, आप खोज सकते हैं कि हम ट्यूटोरियल भाग 8 bis में प्रोटोकॉल के साथ प्‍लान का उपयोग कैसे करते हैं!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)

### GitHub पर स्टार PySyft

हमारे समुदाय की मदद करने का सबसे आसान तरीका सिर्फ िपॉजिटरी को अभिनीत करना है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [स्टार Pysyft](https://github.com/OpenMined/PySyft)

### GitHub पर हमारे ट्यूटोरियल चुनना !

हमने फेडरेटेड और प्राइवेसी-प्रिजर्विंग लर्निंग की बेहतर समझ प्राप्त करने के लिए वास्तव में अच्छा ट्यूटोरियल बनाया और ऐसा होने के लिए हम ईंटों का निर्माण कर रहे हैं।

- [PySyft ट्यूटोरियल्स को चेक करें](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### हमारे Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है! 
- [slack.openmined.org में शामिल हों](http://slack.openmined.org)

### एक कोड परियोजना में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! यदि आप "one off" मिनी-प्रोजेक्ट्स शुरू करना चाहते हैं, तो आप PySyft GitHub जारी करने वाले पृष्ठ पर जा सकते हैं और `Good First Issue` के रूप में चिह्नित मुद्दों की खोज कर सकते हैं।

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्च जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

[OpenMined का ओपन कलेक्टिव पेज](https://opencollective.com/openmined)